In [1]:
from scraping import get_pipeline, _soupage, scrap, get_pays, get_page_pays
from bs4 import BeautifulSoup as BS
import polars as pl
import re
from cleaning import clean_pipeline
import numpy as np

soupe = get_pipeline("https://www.cuisineaz.com/cuisine-du-monde-p282")

In [8]:
df = pl.read_json(r"data\recette.json")

In [9]:
df_clean = clean_pipeline(df)

In [11]:
df_clean

nom,temps,niveau,cout,nb_comment,ingredient_nom,ingredient_qte,img,recette,comment,pays
str,i64,str,f64,i64,list[str],list[str],str,list[str],list[str],str
"""Alcool de pomm…",300,"""Difficile""",0.77,0,"[""Pomme(s) de terre"", ""Blé"", ""Levure""]","[""10 kg"", ""1 kg"", "" ""]","""https://img.cu…","[""Lavez les pommes de terre et faites-les cuire sans les peler pendant une bonne heure. Jetez ensuite l’eau de cuisson et écrasez bien les pommes de terre. Remettez-les ensuite dans une marmite en ajoutant 20 l d’eau. Mélangez bien et faites chauffer jusqu’à 66°C. Puis ajoutez 1 kg de blé écrasé, mélangez bien, couvrez, et remuez régulièrement pendant 2 heures, jusqu’à ce que la température redescende à 30°C. Les enzymes contenues dans le blé vont décomposer l’amidon des pommes de terre."", ""Filtrez la purée et récupérez le liquide. Ajoutez les levures pour fermenter l’alcool. Remuez bien, et laissez le liquide dans une pièce entre 27 et 30°C. Ce processus prend de 3 à 5 jours."", … ""Enfin, diluez votre alcool de pomme de terre avec de l’eau purifiée pour obtenir le degré d’alcool que vous souhaitez, et buvez avec modération. ""]",[],"""allemagne"""
"""Biscuits allem…",52,"""Facile""",0.72,9,"[""Farine"", ""Cassonade"", … ""Blanc(s) d'œuf(s)""]","[""170 g"", ""85 g"", … ""1 ""]","""https://img.cu…","[""Préchauffez le four th.6 (180°C). Dans un saladier, mettez la farine, le gingembre, la cannelle et le mélange d'épices. Ajoutez le beurre en morceaux et travaillez la pâte du bout des doigts jusqu'à ce que le mélange soit de consistance sableuse. "", ""Incorporez le sucre cassonade. Mélangez le sirop de sucre de canne, le lait et le jaune d'oeuf dans un bol. Versez dans le saladier et remuez jusqu'à obtention d'une pâte molle. Posez la pâte sur un plan de travail fariné et pétrissez-la délicatement pendant quelques secondes pour la rendre homogène. "", … ""Pour le glaçage, incorporez dans un bol, peu à peu, le sucre glace au blanc d'oeuf afin d'obtenir un liquide épais. Versez cette préparation dans une poche à douille et nappez les biscuits de glaçage en réalisant la décoration de votre choix. Conservez ces biscuits allemands dans une boîte hermétique.""]","["" SUPER! A part petite panne de gaz juste au moment de la cuisson.....ils ont bien supporté d 'attendre au frigo pendant une nuit.... Merci ! "", "" C'est facile et beau ,trop cool "", … "" Mmmm trop bon, je l'ai faite ! ""]","""allemagne"""
"""Biscuits bretz…",35,"""Facile""",0.4,1,"[""Farine"", ""Lait"", … ""Gros sel""]","[""500 g"", ""300 ml"", … "" ""]","""https://img.cu…","[""Dans un grand saladier, mélangez la farine, la levure et 1,5 cuillères de sel."", ""Ajoutez le beurre et le lait puis pétrissez jusqu'à obtenir une boule de pâte lisse non collante."", … ""Laissez refroidir sur une grille et conservez les bretzels dans une boîte hermétique jusqu'à la dégustation.""]","["" bonsoir, en tant qu'Alsacienne je voudrais vous dire que ce ne sont pas ces bretzels qui vont avec cette recette mais ceux que l'on achète en grandes surfaces !!!! ""]","""allemagne"""
"""Biscuits de No…",30,"""Facile""",0.75,1,"[""Farine"", ""Amande concassée"", … ""Beurre""]","[""250 g"", ""100 g"", … ""50 g""]","""https://img.cu…","[""Dans un saladier, mélangez la farine, le sucre et le miel. "", ""Ajoutez l'oeuf et mélangez bien. "", … ""Laissez sécher plusieurs heures. ""]","["" Pas assez de précisons dans la recette, les écorces doivent elles être coupées tt petit? Le chocolat est bien trop epais pour mettre sur les biscuits si on suit cette recette....1 seul oeuf ne suffit pas, sinon la pâte n'est que de la poudre... ""]","""allemagne"""
"""Boule de berli…",15,"""Intermédiaire""",0.71,0,"[""Farine"", ""Levure boulangère"", … ""Beurre""]","[""300 g"", ""1 Sachet"", … ""50 g""]","""https://img.cu…","[""Pour la pâte à beignets : Mélangez la farine et la levure dans un grand saladier. "", ""Versez-y le sel et le sucre, puis remuez à nouveau. "", … ""Mêlez et filmez au contact. ""]",[],"""allem

df_clean.write_csv(r"data/recette.csv", separator=",")